In [1]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from stockpy.neural_network import LSTMClassifier, MLPClassifier, CNNClassifier

# read CSV file and drop missing values
# df = pd.read_csv('../stock/AAPL_cat.csv', parse_dates=True, index_col='Date').dropna(how="any")
df = pd.read_csv("../../dataset/ParisHousing.csv").dropna(how='any')
# split data into training and test set
X = df.drop(columns=['cityPartRange'])
y = df['cityPartRange']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
input_size = X_train.shape[1]
output_size = y_train.nunique()
# create model instance and fit to training data
predictor = CNNClassifier(input_size=input_size, 
                          hidden_size=32, 
                          output_size=output_size)
predictor.fit(X=X_train, y=y_train, patience=10, batch_size=24, epochs=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

No improvement after 10 epochs. Stopping early.37499999999996, F1_score 26.1278377577661481694
